In [115]:
import pandas as pd
import os
import numpy as np

#assumed directory structure......(relevent for data access)
#os.listdir()

In [116]:
def us_case_data(doc): 
    
    jhu_header='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data' 
    current_data='/csse_covid_19_time_series/'
    us_domestic='time_series_covid19_confirmed_US.csv'
    url= jhu_header+current_data+us_domestic
    
    df= pd.read_csv(url)
    df.fillna("territory", inplace=True)   #fill in data for non-states trimming!

    bad_rows= df[ (df['Lat'] == 0) & (df['Long_'] == 0) ].index    #drop rows with no location 
    df.drop(bad_rows, axis=0, inplace=True) 
    df.drop(columns=['UID','iso2','iso3','code3'], axis=1, inplace=True)

    df.to_csv(doc, index=False)
    return df    

In [117]:
# assumes in the folder select all uptodate population data
# issue with state populations ie new york so lets parse those out 
def us_population_data(doc): 
    
    data_path='co-est2019-alldata.csv' 
    df = pd.read_csv(data_path)

    toKeep=['STNAME','CTYNAME']

    for i in df.keys():
        if ("2019" in i):
            toKeep.append(i)
    Df=df[toKeep]
    
    toDrop=[]
    for i in range(0,(Df.shape)[0]):
        if(Df['STNAME'][i] == Df['CTYNAME'][i]):
            toDrop.append(i)

    #states=Df.loc[toDrop]   
    #Dc included twice we want it once
    toDrop.remove(327)
    Df.drop(toDrop, axis=0, inplace=True)
    Df.to_csv(doc, index=False)
    
    return Df

In [118]:
# need to remove the state from the geo disp data
def us_demographic_data(doc):
    folder_path='PEP_2018_PEPAGESEX\\'
    metadata_path='PEP_2018_PEPAGESEX_metadata.csv' 
    path=folder_path+metadata_path
    
    df = pd.read_csv(path)
    df.drop(columns=(df.keys()[1]),inplace=True)
    df1=df.to_numpy()
    toKeep=['GEO.display-label']
    for i in df1:
        if ("2018" in i[0]):
            toKeep.append(i[0])

    data_path='PEP_2018_PEPAGESEX_with_ann.csv'
    path=folder_path+data_path
    
    dat_df = pd.read_csv(path)
    Df=dat_df[toKeep]
    Df.drop([0], axis=0, inplace=True)
    Df.to_csv(doc, index=False)
    
    return Df

In [119]:
def assemble_us_data(docs, print_tag=False):
    Df1=us_case_data(docs[0])
    Df2=us_population_data(docs[1])
    Df3=us_demographic_data(docs[2])
    
    if(print_tag):
        print(Df1.keys())
        print(Df2.keys())
        print(Df3.keys())

In [120]:
#grab all known data about an area if fips is not -1 then remap it to a real name 
# add fips later on
# clean after test
def grab_county_data(csv_names, name, fips =-1,print_tag=False):
   
    csv_key_cols=['Admin2','CTYNAME','GEO.display-label']

    ret=pd.DataFrame([])
    for i in range(0, len(csv_names)):
        df=pd.read_csv(csv_names[i])
        Df=df.loc[df[csv_key_cols[i]].str.contains(name, case=False)]
        ret=pd.concat([ret, Df], axis=1)
        if(print_tag):
            print(Df.shape)
        
    ret.drop(columns=['STNAME', 'CTYNAME','GEO.display-label'], axis=1, inplace=True)   
    return ret

In [121]:
# add options to grab county data for subsets its alot otherwise...
def grab_case_data(csv_names, name, fips =-1):

    df=pd.read_csv(csv_names[0])
    Df=df.loc[df['Admin2'].str.contains(name, case=False)]
    Df.drop(columns=['FIPS', 'Admin2', 'Province_State', 'Country_Region', 'Lat', 'Long_',
       'Combined_Key'], axis=1, inplace=True)
    
    return Df

In [122]:
def grab_cnty_pop(csv_names, name, fips=-1, state=False):
    df=pd.read_csv(csv_names[1])
    Df=df.loc[df['CTYNAME'].str.contains('new york', case=False)]
    
    return np.array(Df['POPESTIMATE2019'])[0]

In [124]:
#tests
#csv_names= ['US_case_data.csv','US_pop_data.csv','us_demo_data.csv']
#assemble_us_data(csv_names) 

#grab_cnty_pop(csv_names, 'New york')


1628706